In [ ]:
import pandas as pd
from pandas.api.types import CategoricalDtype
from pandas.plotting import table
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import seaborn as sns
import six
from scipy.stats import chisquare

In [ ]:
df_furtos = pd.read_csv('D:\\EPS\\TCC\\data_cleaned\\SSP\\furtos_2018.csv', sep=",", decimal=".")

In [ ]:
furtos_2018_grp = df_furtos.groupby(['DIASEMANA', 'DATAOCORRENCIA'])['DATAOCORRENCIA'].count().rename('TOTAL').reset_index()

In [ ]:
furtos_2018_grp

In [ ]:
furtos_2018_grp['DIASEMANA'] = furtos_2018_grp['DIASEMANA'].map({0: 'Segunda', 1: 'Terca', 2: 'Quarta', 3: 'Quinta', 4: 'Sexta', 5: 'Sabado', 6: 'Domingo'})

In [ ]:
df = furtos_2018_grp.rename({'DIASEMANA': 'DAY_WEEK', 'DATAOCORRENCIA': 'DATE', 'TOTAL': 'NUMBER_OF_OCCURRENCES'}, axis=1)

In [ ]:
df['DAY_WEEK'] = df['DAY_WEEK'].map({'Segunda': 'Monday', 'Terca': 'Tuesday', 'Quarta': 'Wednesday', 'Quinta': 'Thursday', 'Sexta': 'Friday', 'Sabado': 'Saturday', 'Domingo': 'Sunday'})

In [ ]:
df.sort_values('DAY_WEEK', inplace=True)

In [ ]:
df

In [ ]:
df1 = df.groupby(['DAY_WEEK'])['NUMBER_OF_OCCURRENCES'].sum().rename('TOTAL').reset_index()

In [ ]:
df1

In [ ]:
df2 = df.groupby(['DAY_WEEK'])['DAY_WEEK'].count().rename('QTD').reset_index()

In [ ]:
df2

In [ ]:
df3 = df1.merge(df2, how='left', on=['DAY_WEEK'])

In [ ]:
df3

In [ ]:
df3['LAMBDA'] = df3['TOTAL']/df3['QTD']

In [ ]:
df3

In [ ]:
df3.sort_values('DAY_WEEK', inplace=True)

In [ ]:
df3

In [ ]:
x = []
x=np.array(x)

for index, item in df3.iterrows():
    np.random.seed(42)
    y = np.random.poisson(item['LAMBDA'], item['QTD'])
    x = np.concatenate((x, y))

In [ ]:
x

In [ ]:
x.size

In [ ]:
df['expected_value'] = x.tolist()

In [ ]:
df

In [ ]:
chi_square = []

p_value = []

for index, item in df3.iterrows():
    df_day = df.loc[df['DAY_WEEK'] == item['DAY_WEEK']]
    
    chi_square.append(chisquare(f_obs = df_day['NUMBER_OF_OCCURRENCES'], f_exp = df_day['expected_value'])[0])
    p_value.append(chisquare(f_obs = df_day['NUMBER_OF_OCCURRENCES'], f_exp = df_day['expected_value'])[1])

In [ ]:
chi_square = np.array(chi_square)
p_value = np.array(p_value)

In [ ]:
chi_square

In [ ]:
p_value

In [ ]:
df3['chi_square'] = chi_square.tolist()
df3['p_value'] = p_value.tolist()

In [ ]:
df3

In [ ]:
df3.rename({'DAY_WEEK': 'DIASEMANA'}, axis=1, inplace=True)
df3

In [ ]:
df3['DIASEMANA'] = df3['DIASEMANA'].map({'Monday': 'Segunda', 'Tuesday': 'Terca', 'Wednesday': 'Quarta', 'Thursday': 'Quinta', 'Friday': 'Sexta', 'Saturday': 'Sabado', 'Sunday': 'Domingo'})
df3

In [ ]:
cats = ['Segunda', 'Terca', 'Quarta', 'Quinta', 'Sexta', 'Sabado', 'Domingo']
cat_type = CategoricalDtype(categories=cats, ordered=True)
df3['DIASEMANA'] = df3['DIASEMANA'].astype(cat_type)
df3.sort_values("DIASEMANA", inplace=True)
df3

In [ ]:
def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')

    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in  six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    fig.savefig('qui-quadrado_resultados_furtos_2018.png')
    
    return ax

render_mpl_table(df3, header_columns=0, col_width=4.0)